# Blue Brain BioExplorer
![](../../bioexplorer_banner.png)

# Low Glucose Scenario

## Connect to BioExplorer backend

In [33]:
from bioexplorer import BioExplorer, Protein, Surfactant, Membrane, Cell, Sugars, \
                        Volume, AnimationParams, Vector2, Vector3, Quaternion
import nglview

hostname = 'localhost:5000'
resource_folder = '../../../tests/test_files/'
    
be = BioExplorer(hostname)
core = be.core_api()
version = be.version()
print('Version: ' + version)

export_to_cache = False

scenario = 'low_glucose'

Version: 1.1.2


In [34]:
'''Accelerate loading by not showing models as they are loaded'''
status = be.set_general_settings(
    logging_enabled=False,
    model_visibility_on_creation=False)
status = core.set_renderer(samples_per_pixel=1, subsampling=1, max_accum_frames=1)
be.reset_scene()

{'contents': 'Removed 152 models', 'status': True}

## Global parameters

In [35]:
# Scene
scene_size = Vector3(800.0, 800.0, 800.0)

# Proteins
protein_radius_multiplier = 2.0
protein_representation = be.REPRESENTATION_ATOMS_AND_STICKS
# protein_representation = be.REPRESENTATION_CONTOURS

# Glycans
# add_glycans = True
add_glycans = False
glycan_representation = be.REPRESENTATION_ATOMS_AND_STICKS
# glycan_representation = be.REPRESENTATION_CONTOURS

# Viruses
nb_protein_s = 62
nb_protein_e = 42
nb_protein_m = 50
add_rna = False

# Immune system
nb_glucoses = 120000
nb_lactoferrins = 150
nb_defensins = 300
nb_defensins_on_virus = 2

# Cell
cell_name = 'Cell'
cell_position = Vector3(4.5, -186.0, 7.0)
cell_size = Vector3(scene_size.x, scene_size.y / 10.0, scene_size.z)
cell_nb_receptors = 100

## Resources

In [36]:
cache_folder = resource_folder + 'caches/' + version + '/'
image_folder = resource_folder + 'images/'
pdb_folder = resource_folder + 'pdb/'
rna_folder = resource_folder + 'rna/'
obj_folder = resource_folder + 'obj/'
glycan_folder = pdb_folder + 'glycans/'
membrane_folder = pdb_folder + 'membrane/'

complex_paths = [glycan_folder + 'complex/33.pdb', glycan_folder + 'complex/34.pdb',
                 glycan_folder + 'complex/35.pdb',glycan_folder + 'complex/36.pdb']
high_mannose_paths = [glycan_folder + 'high-mannose/1.pdb', 
                      glycan_folder + 'high-mannose/2.pdb',
                      glycan_folder + 'high-mannose/3.pdb',
                      glycan_folder + 'high-mannose/4.pdb']
hybrid_paths = [glycan_folder + 'hybrid/24.pdb']
o_glycan_paths = [glycan_folder + 'o-glycan/12.pdb']

glucose_path = pdb_folder + 'glucose.pdb'
lactoferrin_path=pdb_folder + 'immune/1b0l.pdb'
defensin_path = pdb_folder + 'immune/1ijv.pdb'

surfactant_head_source = pdb_folder + 'surfactant/1pw9.pdb'
surfactant_branch_source = pdb_folder + 'surfactant/1k6f.pdb'

lymphocyte_path = obj_folder + 'lymphocyte.obj'

## Coronaviruses

In [ ]:
nglview.show_file(pdb_folder + 'sars-cov-2-v1.pdb')

In [ ]:
nglview.show_file(pdb_folder + '6vyb.pdb')

In [ ]:
nglview.show_file(pdb_folder + 'QHD43419a.pdb')

In [ ]:
nglview.show_file(pdb_folder + 'QHD43418a.pdb')

In [37]:
be.add_coronavirus(
    name='Coronavirus 1', resource_folder=resource_folder,
    position=Vector3(-5.0, 19.0, -36.0), 
    rotation=Quaternion(0.0, 1.0, 0.0, 0.0),
    add_glycans=add_glycans,
    atom_radius_multiplier=protein_radius_multiplier,
    representation=protein_representation
)
be.add_coronavirus(
    name='Coronavirus 2', resource_folder=resource_folder,
    position=Vector3(73.0, 93.0, -115.0), 
    rotation=Quaternion(0.0, 0.0, 1.0, 0.0),
    add_glycans=add_glycans,
    atom_radius_multiplier=protein_radius_multiplier,
    representation=protein_representation
)
be.add_coronavirus(
    name='Coronavirus 3', resource_folder=resource_folder,
    position=Vector3(-84.0, 110.0, 75.0), 
    rotation=Quaternion(0.707, 0.707, 0.0, 0.0),
    add_glycans=add_glycans,
    atom_radius_multiplier=protein_radius_multiplier,
    representation=protein_representation
)
be.add_coronavirus(
    name='Coronavirus 4', resource_folder=resource_folder,
    position=Vector3(-74.9, -97.1, 228.8),
    add_glycans=add_glycans,
    atom_radius_multiplier=protein_radius_multiplier,
    representation=protein_representation
)
be.add_coronavirus(
    name='Coronavirus 5', resource_folder=resource_folder,
    position=Vector3(200.0, 20.0, -150.0), 
    rotation=Quaternion(0.0, 0.707, 0.707, 0.0),
    add_glycans=add_glycans,
    atom_radius_multiplier=protein_radius_multiplier,
    representation=protein_representation
)

In [ ]:
if export_to_cache:
    '''Show loaded models'''
    status = be.set_models_visibility(True)
    '''Set materials'''
    be.apply_default_color_scheme(shading_mode=be.SHADING_MODE_BASIC)
    '''Export to cache file'''
    cache_filename = cache_folder + scenario + '_viruses.bioexplorer'
    if add_rna:
        cache_filename = cache_folder + scenario + '_open_viruses.bioexplorer'
    print(cache_filename)
    be.export_to_file(cache_filename)

## Host cell

In [ ]:
def add_cell(
        name, size, height, nb_receptors,
        position=Vector3(), random_seed=1):
    
    ace2_receptor = Protein(
        sources=[pdb_folder + '6m18.pdb'], occurences=nb_receptors,
        position=Vector3(0.0, 6.0, 0.0))
    
    membrane = Membrane(
        lipid_sources=[
            membrane_folder + 'segA.pdb',
            membrane_folder + 'segB.pdb',
            membrane_folder + 'segC.pdb',
            membrane_folder + 'segD.pdb'
        ]
    )

    cell = Cell(
        name=name, size=size, extra_parameters=[height],
        shape=be.ASSEMBLY_SHAPE_SINUSOID, 
        membrane=membrane, receptor=ace2_receptor,
        random_position_seed=1, random_position_strength=0.025,
        random_rotation_seed=2, random_rotation_strength=2.0
    )
    
    be.add_cell(
        cell=cell, position=position,
        representation=protein_representation,
        random_seed=random_seed)

    if nb_receptors != 0 and add_glycans:
        be.add_multiple_glycans(
            representation=glycan_representation, assembly_name=name, 
            glycan_type=be.NAME_GLYCAN_COMPLEX,
            protein_name=be.NAME_RECEPTOR, paths=complex_paths, 
            indices=[53, 90, 103, 322, 432, 690])
        be.add_multiple_glycans(
            representation=glycan_representation, assembly_name=name,
            glycan_type=be.NAME_GLYCAN_HYBRID,
            protein_name=be.NAME_RECEPTOR, paths=hybrid_paths, 
            indices=[546])

        indices = [[155, Quaternion(0.707, 0.0, 0.707, 0.0)],
                   [730, Quaternion(0.707, 0.0, 0.707, 0.0)]]
        for index in indices:
            o_glycan_name = name + '_' + be.NAME_GLYCAN_O_GLYCAN + '_' + str(index[0])
            o_glycan = Sugars(
                assembly_name=name, name=o_glycan_name, source=o_glycan_paths[0],
                protein_name=name + '_' + be.NAME_RECEPTOR, representation=glycan_representation,
                chain_ids=[2, 4], site_indices=[index[0]], rotation=index[1])
            be.add_sugars(o_glycan)

In [ ]:
cell_position = Vector3(4.5, -186.0, 7.0)
cell_height = scene_size.y / 10.0
add_cell(
    name='Cell', nb_receptors=cell_nb_receptors,
    size=Vector2(scene_size.x * 2.0, scene_size.z * 2.0), height=cell_height,
    position=cell_position, random_seed=10)

In [ ]:
if export_to_cache:
    '''Show loaded models'''
    status = be.set_models_visibility(True)
    '''Set materials'''
    be.apply_default_color_scheme(shading_mode=be.SHADING_MODE_BASIC)
    be.set_protein_color_scheme(
        cell_name, cell_name + '_' + be.NAME_RECEPTOR,
        be.COLOR_SCHEME_CHAINS, 'OrRd_r', 7)
    '''Export to cache file'''
    cache_filename = cache_folder + 'cell.bioexplorer'
    print(cache_filename)
    be.export_to_file(cache_filename)

## Immune system

### Surfactant A and D

In [ ]:
nglview.show_file(surfactant_head_source)

In [38]:
def add_surfactant_d(name, position, animation_params):
    surfactant_d = Surfactant(
        name=name, surfactant_protein=be.SURFACTANT_PROTEIN_D, 
        head_source=surfactant_head_source,
        branch_source=surfactant_branch_source)
    be.add_surfactant(
        surfactant=surfactant_d, representation=protein_representation, 
        position=position, animation_params=animation_params)

add_surfactant_d(
    name='Surfactant-D 1', 
    position=Vector3(74.0,  24.0, -45.0),
    animation_params=AnimationParams(1))
add_surfactant_d(
    name='Surfactant-D 2',
    position=Vector3(-11.0,  103.0, 20.0),
    animation_params=AnimationParams(1))
add_surfactant_d(
    name='Surfactant-D 3',
    position=Vector3(-165.0, 140.0, 105.0),
    animation_params=AnimationParams(1))
add_surfactant_d(
    name='Surfactant-D 4',
    position=Vector3(-260.0,  50.0, 150.0),
    animation_params=AnimationParams(2))

In [39]:
def add_surfactant_a(name, position, random_seed):
    surfactant_a = Surfactant(
        name=name, surfactant_protein=be.SURFACTANT_PROTEIN_A, 
        head_source=surfactant_head_source,
        branch_source=surfactant_branch_source)
    be.add_surfactant(
        surfactant=surfactant_a, representation=protein_representation, 
        position=position, random_seed=random_seed)

add_surfactant_a(
    name='Surfactant-A 1', 
    position=Vector3(200.0,  50.0, 150.0),
    random_seed=2)

TypeError: add_surfactant() got an unexpected keyword argument 'random_seed'

### Glucose

In [ ]:
nglview.show_file(glucose_path)

In [ ]:
glucose = Protein(
    sources=[glucose_path], load_non_polymer_chemicals=True, 
    occurences=nb_glucoses)
volume = Volume(
    name=be.NAME_GLUCOSE, size=scene_size, protein=glucose)
status = be.add_volume(
    volume=volume, representation=protein_representation,
    position=Vector3(0.0, scene_size / 2.0 - 200.0, 0.0))

### Lactoferrins

In [ ]:
nglview.show_file(lactoferrin_path)

In [ ]:
lactoferrin = Protein(
    sources=[lactoferrin_path], load_non_polymer_chemicals=True,
    occurences=nb_lactoferrins)
lactoferrins_volume = Volume(
    name=be.NAME_LACTOFERRIN, size=scene_size, protein=lactoferrin)
status = be.add_volume(
    volume=lactoferrins_volume, representation=protein_representation,
    position=Vector3(0.0, scene_size / 2.0 - 200.0, 0.0),
    random_seed=2)

### Defensins

In [ ]:
nglview.show_file(defensin_path)

In [ ]:
defensin = Protein(
    sources=[defensin_path], load_non_polymer_chemicals=True, 
    occurences=nb_defensins)
defensins_volume = Volume(
    name=be.NAME_DEFENSIN, size=scene_size, protein=defensin)
status = be.add_volume(
    volume=defensins_volume, representation=protein_representation,
    position=Vector3(0.0, scene_size / 2.0 - 200.0, 0.0),
    random_seed=3)

### Lymphocyte

In [ ]:
name = 'Emile'
protein_sources = [
    membrane_folder + 'segA.pdb',
    membrane_folder + 'segB.pdb',
    membrane_folder + 'segC.pdb',
    membrane_folder + 'segD.pdb'
]

clip_planes = [
    [1.0, 0.0, 0.0, scene_size.x / 1.5 + 5],
    [-1.0, 0.0, 0.0, scene_size.x / 2.0 + 5],
    [0.0, 1.0, 0.0, 250.0],
    [0.0, -1.0, 0.0, 550.0]
]

mesh_source = obj_folder + 'clipped_lymphocyte.obj'

position = Vector3(-1430.0, 100.0, 30.0)
rotation = Quaternion(0.707, 0.707, 0.0, 0.0)
scale = Vector3(1.0, 1.0, 1.0)

mesh_based_membrane = Membrane(
    mesh_source=mesh_source, protein_sources=protein_sources, 
    density=5.0, surface_variable_offset=5.0)

status = be.add_mesh_based_membrane(
    name, mesh_based_membrane, position=position,
    rotation=rotation, scale=scale, clipping_planes=clip_planes)

for i in range(len(protein_sources)):
    status = be.set_protein_color_scheme(
        assembly_name=name, name=be.NAME_MEMBRANE + '_' + str(i),
        color_scheme=be.COLOR_SCHEME_CHAINS,
        palette_name='OrRd', palette_size=5)

### Materials

In [ ]:
be.apply_default_color_scheme(
    shading_mode=be.SHADING_MODE_DIFFUSE,
    specular_exponent=50)

In [ ]:
'''Show loaded models'''
status = be.set_models_visibility(True)

In [ ]:
if export_to_cache:
    '''Export to cache file'''
    cache_filename = cache_folder + scenario + '_immune.bioexplorer'
    print(cache_filename)
    be.export_to_file(cache_filename)

## Export full scene to cache file

In [ ]:
if export_to_cache:
    cache_filename = cache_folder + scenario + '_scenario.bioexplorer'
    print(cache_filename)
    be.export_to_file(cache_filename)

## Rendering settings

In [ ]:
status = be.set_rendering_quality(be.RENDERING_QUALITY_HIGH)